In [6]:
import pandas as pd 
from feature_selection import *
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
from dataset import make_dataset
from os.path import join
from tensorflow import keras

In [36]:
train_path = join('jeju_data', 'train_new.parquet')
test_path = join('jeju_data', 'test_new.parquet')

X, y, test = make_dataset(train_path, test_path)

100%|██████████| 291241/291241 [00:02<00:00, 124354.22it/s]


In [48]:
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=1103)

In [13]:
# train = pd.read_parquet('./jeju_data/train_new.parquet')
# test = pd.read_parquet('./jeju_data/test_new.parquet')
# str_col = ['day_of_week','start_turn_restricted','end_turn_restricted']
# for i in str_col:
#     le = LabelEncoder()
#     le=le.fit(train[i])
#     train[i]=le.transform(train[i])
    
#     for label in np.unique(test[i]):
#         if label not in le.classes_: 
#             le.classes_ = np.append(le.classes_, label)
#     test[i]=le.transform(test[i])
# y1 = train['target'] 

# X1 = train.drop(['id', 'target','road_name', 'start_node_name', 'end_node_name','vehicle_restricted'], axis=1)

# test1 = test.drop(['id', 'road_name', 'start_node_name', 'end_node_name','vehicle_restricted'], axis=1)


# x_train, x_val, y_train, y_val = train_test_split(X1, y1, test_size=0.2, random_state=1103)

In [49]:
split_fraction = 0.715
train_split = int(split_fraction * int(X1.shape[0]))
step = 6

past = 720
future = 72
learning_rate = 0.001
batch_size = 256
epochs = 10

In [50]:
start = past + future
end = start + train_split
sequence_length = int(past / step)

In [51]:
dataset_train = keras.preprocessing.timeseries_dataset_from_array(
    x_train,
    y_train,
    sequence_length=sequence_length,
    sampling_rate=step,
    batch_size=batch_size,
)

In [52]:
dataset_val = keras.preprocessing.timeseries_dataset_from_array(
    x_val,
    y_val,
    sequence_length=sequence_length,
    sampling_rate=step,
    batch_size=batch_size,
)

In [53]:
for batch in dataset_train.take(1):
    inputs, targets = batch
print("Input shape:", inputs.numpy().shape)
print("Target shape:", targets.numpy().shape)

Input shape: (256, 120, 21)
Target shape: (256,)


In [54]:
inputs = keras.layers.Input(shape=(inputs.shape[1], inputs.shape[2]))
lstm_out = keras.layers.LSTM(32)(inputs)
outputs = keras.layers.Dense(1)(lstm_out)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss="mae")
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 120, 21)]         0         
                                                                 
 lstm_2 (LSTM)               (None, 32)                6912      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 6,945
Trainable params: 6,945
Non-trainable params: 0
_________________________________________________________________


In [67]:
data = train.sort_values(['base_date', 'base_hour'])
copy_data = data.copy()
copy_data.set_index('id', inplace=True)
copy_data


,base_date,day_of_week,base_hour,lane_count,road_rating,road_name,multi_linked,connect_code,maximum_speed_limit,vehicle_restricted,...,road_type,start_node_name,start_latitude,start_longitude,start_turn_restricted,end_node_name,end_latitude,end_longitude,end_turn_restricted,target
id,,,,,,,,,,,,,,,,,,,,,
TRAIN_0002885,20210901,2,0,3,103,일반국도11호선,0,0,70.0,0.0,...,0,인다마을,33.471061,126.545467,0,염광아파트,33.469352,126.547314,0,39.0
TRAIN_0005183,20210901,2,0,1,107,-,0,0,30.0,0.0,...,0,삼다식품,33.429714,126.328707,0,어도초등교,33.434581,126.324312,0,40.0
TRAIN_0005280,20210901,2,0,1,106,지방도1115호선,0,0,60.0,0.0,...,0,제4산록교,33.290116,126.489006,0,제5산록교,33.291731,126.504396,0,46.0
TRAIN_0008728,20210901,2,0,2,103,일반국도12호선,0,0,80.0,0.0,...,0,종달교차로,33.496887,126.893155,0,종달1교차로,33.492178,126.894049,1,61.0
TRAIN_0016926,20210901,2,0,2,106,번영로,0,0,70.0,0.0,...,0,와흘교차로,33.470925,126.645640,1,회천교차로,33.475256,126.637239,1,76.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TRAIN_4618704,20220731,4,22,1,107,-,0,0,30.0,0.0,...,3,천제이교,33.245860,126.417641,0,천제이교,33.244882,126.419089,0,45.0
TRAIN_4645714,20220731,4,22,1,106,지방도1118호선,0,0,60.0,0.0,...,0,제주삼다수,33.424343,126.675832,0,육성목장,33.421793,126.676661,0,75.0
TRAIN_4658541,20220731,4,22,2,103,일반국도16호선,0,0,30.0,0.0,...,3,제2서홍교,33.262690,126.542300,0,제2서홍교,33.262662,126.542060,0,39.0


In [55]:
model.fit(
    dataset_train,
    epochs=epochs,
    validation_data=dataset_val)

Epoch 1/10
14689/14689 [==============================] - 175s 12ms/step - loss: 14.6781 - val_loss: 13.2143
Epoch 2/10
14689/14689 [==============================] - 159s 11ms/step - loss: 13.2261 - val_loss: 13.2142
Epoch 3/10
13861/14689 [===========================>..] - ETA: 7s - loss: 13.2251

KeyboardInterrupt: 